In [1]:
%matplotlib inline

import math
import datetime

import numpy as np

from pandas import DataFrame
from pandas import concat

import tensorflow as tf
from tensorflow import keras

from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams['font.family'] = 'serif'

debug = True

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
cuda = False #FIXME: re-install tensorflow and CUDA to get it to work again
debug = True
N = 10
one_hot_size = 32

In [3]:
#MATLAB colors
colors = [[0, 0.4470, 0.7410],
          [0.8500, 0.3250, 0.0980],
          [0.9290, 0.6940, 0.1250],
          [0.4940, 0.1840, 0.5560],
          [0.4660, 0.6740, 0.1880],
          [0.3010, 0.7450, 0.9330],
          [0.6350, 0.0780, 0.1840]];

colormap = plt.get_cmap('rainbow')

def getColor(num, brightness=1):
    col = colors[num]
    return [col[0]*brightness, col[1]*brightness, col[2]*brightness]

## Encoding/decoding ThingML messages into 1-D tensors

### 1-hot encoding

ThingML messages are encoded into 1D tensors, using a simple 1-hot encoding:
- first, we encode the message type. Here, indices 0..13 are used to encode messages `clear`..`updateIA`. In principle, there will only be a single `1.0` value in the range 0..13, indicating that a message is either a `clear` message or (XOR) a `setColor` message, etc. However, the prediction will likely produce result like `clear:0.87`, `setColor:0.14`, etc. We decode the message type by taking the highest score.
- second, we encode the parameters of the message e.g. indices 14..16 are used to encode `r`, `g` and `b`. Note that different messages can have the same parameters (or a common subset), and in this case, indices for parameters will be reused. For example, indices 14..16 are used by `setColor` and `setBGColor`. A parameter is predicted present if it scores `>= 0.5`. 

The encoding of training, validation and test data is based by parsing lines starting with `!` and `@` in the logs.

### Example

A message `setColor(r=107, b=94, b=174)` send by the `BreakoutGame` thing is serialized as `[3,107,94,174]` on the network. 

In the log files, this is represented by `!3,107,94,174,` followed by `@BreakoutGameNodeJS@setColor,r,g,b,`. See `def logToData(filename)`

The same message is 1-hot encoded as `[0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 1.0, 1.0, 0, 0, ..., 0]`. It is decoded as `[setColor, r, g, b]`.

Assume a prediction producing the following 1D tensor:

`predict = [0.22, 0.87, 0.17, 0.32, 0.12, 0.33, 0.22, 0.13, 0.17, 0.16, 0.32, 0.33, 0.37, 0.18, 0.67, 0.44, 0.55, <0.5, <0.5, ..., <0.5]`

This will be decoded as `[setColor, r, b]`. Indeed:
- predict[1] > predict[*n*] for *n* in 0..13, and predict[1] encodes for `setColor`
- predict[14] = 0.67 >= 0.5, and predict[14] encodes for parameter `r`
- predict[15] = 0.44 < 0.5,  and predict[15] encodes for parameter `g`. Parameter `g` is thus not deemed to be present.
- predict[16] = 0.55 >= 0.5, and predict[16] encodes for parameter `b`

In [4]:
# 1-hot encoding of the ThingML messages between controller/AI and game logic
def encode(msg):
    encoded = [0]*one_hot_size
    valid = False
    for x in msg:
        # Encode message
        if x.startswith('clear'):
            encoded[0] = 1
            valid = True
        elif x.startswith('setColor'):
            encoded[1] = 1
            valid = True
        elif x.startswith('setBGColor'):
            encoded[2] = 1
            valid = True
        elif x.startswith('drawRect'):
            encoded[3] = 1
            valid = True
        elif x.startswith('fillRect'):
            encoded[4] = 1
            valid = True
        elif x.startswith('drawInteger'):
            encoded[5] = 1
            valid = True
        elif x.startswith('drawThingML'):
            encoded[6] = 1
            valid = True
        elif x.startswith('create'):
            encoded[7] = 1
            valid = True
        elif x.startswith('destroy'):
            encoded[8] = 1
            valid = True
        elif x.startswith('update'):
            encoded[9] = 1
            valid = True
        elif x.startswith('displayReady'):
            encoded[10] = 1
            valid = True
        elif x.startswith('velocity'):
            encoded[11] = 1
            valid = True
        elif x.startswith('position'):
            encoded[12] = 1
            valid = True
        elif x.startswith('updateIA'):
            encoded[13] = 1
            valid = True
        else: # Encode params                       
            if x == 'r':
                encoded[14] = 1
            elif x == 'g':
                encoded[15] = 1
            elif x == 'b':
                encoded[16] = 1  
            elif x == 'x':
                encoded[17] = 1   
            elif x == 'y':
                encoded[18] = 1
            elif x == 'width':
                encoded[19] = 1
            elif x == 'height':
                encoded[20] = 1  
            elif x == 'v':
                encoded[21] = 1   
            elif x == 'digits':
                encoded[22] = 1                
            elif x == 'scale':
                encoded[23] = 1  
            elif x == 'xsize':
                encoded[24] = 1   
            elif x == 'ysize':
                encoded[25] = 1   
            elif x == 'dx':
                encoded[26] = 1                
            elif x == 'dy':
                encoded[27] = 1  
            elif x == 'padx':
                encoded[28] = 1   
            elif x == 'pady':
                encoded[29] = 1                   
            elif x == 'ballx':
                encoded[30] = 1   
            elif x == 'bally':
                encoded[31] = 1
    if valid:
        #print(encoded)
        return encoded
    else:
        return None


def decode(encoded):
    message = []
    
    # decode message
    max_id = -1
    max_v = 0
    for i in range(0, 13):
        if encoded[i] > max_v:
            max_v = encoded[i]
            max_id = i
    if max_id == 0:
        message.append('clear')
    elif max_id == 1:
        message.append('setColor')
    elif max_id == 2:
        message.append('setBGColor')
    elif max_id == 3:
        message.append('drawRect')
    elif max_id == 4:
        message.append('fillRect')
    elif max_id == 5:
        message.append('drawInteger')
    elif max_id == 6:
        message.append('drawThingML')
    elif max_id == 7:
        message.append('create')
    elif max_id == 8:
        message.append('destroy')
    elif max_id == 9:
        message.append('update')
    elif max_id == 10:
        message.append('displayReady')
    elif max_id == 11:
        message.append('velocity')
    elif max_id == 12:
        message.append('position')
    elif max_id == 13:
        message.append('updateIA')
    else:
        message.append('???')
            
    # decode params
    if encoded[14] >= 0.5:
        message.append('r')
    if encoded[15] >= 0.5:
        message.append('g')
    if encoded[16] >= 0.5:
        message.append('b')
    if encoded[17] >= 0.5:
        message.append('x') 
    if encoded[18] >= 0.5:
        message.append('y') 
    if encoded[19] >= 0.5:
        message.append('width') 
    if encoded[20] >= 0.5:
        message.append('height') 
    if encoded[21] >= 0.5:
        message.append('v')   
    if encoded[22] >= 0.5:
        message.append('digits')                 
    if encoded[23] >= 0.5:
        message.append('scale')                 
    if encoded[24] >= 0.5:
        message.append('xsize')                 
    if encoded[25] >= 0.5:
        message.append('ysize')                 
    if encoded[26] >= 0.5:
        message.append('dx')                 
    if encoded[27] >= 0.5:
        message.append('dy')                 
    if encoded[28] >= 0.5:
        message.append('padx')   
    if encoded[29] >= 0.5:
        message.append('pady')              
    if encoded[30] >= 0.5:
        message.append('ballx')   
    if encoded[31] >= 0.5:
        message.append('bally')   
                        
    return message
        

def logToData(filename):
    file = open(filename)
    while True:
        line = file.readline()      
        if not line:
            break        
        if line.startswith('!'):
            raw_bytes = list(line[1:].strip().split(',')[:-1])
            raw_bytes_float = []
            for b in raw_bytes:
                raw_bytes_float.append(int(b)/255)
            while True:
                line = file.readline()      
                if not line:
                    break        
                elif line.startswith('@'):
                    encoded = encode(line.split('@')[2].split(','))
                    if encoded != None:
                        yield raw_bytes_float, encoded
                    break;    

In [5]:
def loadAllData(base, N):
    maxbytelen = 0
    logs = []
    for i in range(0,N):
        log = list(logToData(base+str(i)+'.log'))
        logs.append(log)
        for msg in log:
            length = len(msg[0])
            maxbytelen = max(maxbytelen, length)
    X = []
    Y = []
    for log in logs:
        x = np.zeros((len(log),maxbytelen))
        #x = np.zeros((len(log),one_hot_size))
        y = np.zeros((len(log),one_hot_size))
        for i, msg in enumerate(log):
            x[i,0:len(msg[0])] = msg[0]
            y[i,:] = msg[1]
        X.append(x)
        Y.append(y)
    return X, Y, maxbytelen

In [6]:
base_dir = '../../../../../experiments1-logs/thingml-bytes-logs/nodejs/'

base_X, base_Y, base_maxbytelen = loadAllData(base_dir+'base/nodejs', N)
static_X, static_Y, static_maxbytelen = loadAllData(base_dir+'static/nodejs', N)
runtime_X, runtime_Y, runtime_maxbytelen = loadAllData(base_dir+'dynamic/nodejs', N)

In [7]:
print(str(base_maxbytelen) + ', ' + str(static_maxbytelen) + ', ' + str(runtime_maxbytelen))

9, 8, 10


In [8]:
def split(data, train_ratio = 0.60):
    train = []
    test = []
    for log in data:
        length = log.shape[0]
        train_size = int(length * train_ratio)
        train.append(log[0:train_size,:])
        test.append(log[train_size:,:])
    return train, test

base_X_train, base_X_test = split(base_X)
base_Y_train, base_Y_test = split(base_Y)

static_X_train, static_X_test = split(static_X)
static_Y_train, static_Y_test = split(static_Y)

runtime_X_train, runtime_X_test = split(runtime_X)
runtime_Y_train, runtime_Y_test = split(runtime_Y)

In [9]:
number_of_msgs = 10 

def series_to_supervised(all_data, maxbytelen):
    results = []
    for j in range(0,N):
        result = []
        data = all_data[j].flatten()
        for i in range(0,int(len(data)/(number_of_msgs*maxbytelen))):
            time_window = data[i*maxbytelen:number_of_msgs*maxbytelen+i*maxbytelen]
            result.append(time_window)
        results.append(result)
    np_results = np.array(results)
    np_results = np.reshape(np_results, (np_results.shape[0], np_results.shape[1], number_of_msgs, int(np_results.shape[2]/number_of_msgs)))
    return np_results

base_X_train_supervised = series_to_supervised(base_X_train, base_maxbytelen)
static_X_train_supervised = series_to_supervised(static_X_train, static_maxbytelen)
runtime_X_train_supervised = series_to_supervised(runtime_X_train, runtime_maxbytelen)

base_Y_train_supervised = series_to_supervised(base_Y_train, one_hot_size)
static_Y_train_supervised = series_to_supervised(static_Y_train, one_hot_size)
runtime_Y_train_supervised = series_to_supervised(runtime_Y_train, one_hot_size)

base_X_test_supervised = series_to_supervised(base_X_test, base_maxbytelen)
static_X_test_supervised = series_to_supervised(static_X_test, static_maxbytelen)
runtime_X_test_supervised = series_to_supervised(runtime_X_test, runtime_maxbytelen)

base_Y_test_supervised = series_to_supervised(base_Y_test, one_hot_size)
static_Y_test_supervised = series_to_supervised(static_Y_test, one_hot_size)
runtime_Y_test_supervised = series_to_supervised(runtime_Y_test, one_hot_size)

if debug:
    print(base_X_train_supervised.shape, ' ', base_Y_train_supervised.shape)
    print(static_X_train_supervised.shape, ' ', static_Y_train_supervised.shape)
    print(runtime_X_train_supervised.shape, ' ', runtime_Y_train_supervised.shape)

    print(base_X_test_supervised.shape, ' ', base_Y_test_supervised.shape)
    print(static_X_test_supervised.shape, ' ', static_Y_test_supervised.shape)
    print(runtime_X_test_supervised.shape, ' ', runtime_Y_test_supervised.shape)

(1, 827, 10, 9)   (1, 827, 10, 32)
(1, 1655, 10, 8)   (1, 1655, 10, 32)
(1, 1655, 10, 10)   (1, 1655, 10, 32)
(1, 551, 10, 9)   (1, 551, 10, 32)
(1, 1103, 10, 8)   (1, 1103, 10, 32)
(1, 1103, 10, 10)   (1, 1103, 10, 32)


In [10]:
def keras_model(x, y):    
    model = keras.Sequential()
    #model.add(keras.layers.Dropout(0.2, input_shape=(x,)))
    model.add(keras.layers.TimeDistributed(keras.layers.Dense(200, activation='relu', input_dim=x)))
    if cuda:
        model.add(keras.layers.CuDNNLSTM(100, return_sequences=True))
    else:
        model.add(keras.layers.LSTM(100, return_sequences=True))
    #model.add(keras.layers.Dropout(0.2))
    model.add(keras.layers.TimeDistributed(keras.layers.Dense(50, activation='relu')))
    #model.add(keras.layers.Dropout(0.2))
    model.add(keras.layers.Dense(y, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', 'binary_accuracy'])    
    return model

def train_and_save(model, train_X, train_Y, test_X, test_Y, mode, xp, save=False):        
    earlystop_loss = keras.callbacks.EarlyStopping(monitor='loss', min_delta=0.001, patience=6, verbose=1, mode='min')
    earlystop_val_loss = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.001, patience=6, verbose=1, mode='min')
    earlystop_acc = keras.callbacks.EarlyStopping(monitor='acc', min_delta=0.001, patience=6, verbose=1, mode='max')
    earlystop_val_acc = keras.callbacks.EarlyStopping(monitor='val_acc', min_delta=0.001, patience=6, verbose=1, mode='max')
    
    tensorboard_log = keras.callbacks.TensorBoard(log_dir=base_dir+'/tensorboard/'+mode+'/'+str(xp))
    
    callbacks_list = [tensorboard_log, earlystop_loss, earlystop_val_loss, earlystop_acc, earlystop_val_acc]  
        
    model.fit(train_X, train_Y, batch_size=1, epochs=20, validation_split=0.2, verbose=1, callbacks=callbacks_list)

    if save:
        model.save_weights(base_dir + mode + str(xp) + '_class_lstm.h5')
#         try:
#             keras.utils.plot_model(model, to_file=base_dir + mode + str(xp) + '_class_lstm.png')
#         except:
#             print('Cound not save image for Keras model at ', to_file=base_dir + mode + str(xp)+'_class_lstm.png')
            
#     if debug:
#         result = model.evaluate(test_X, test_Y)
#         print(result)
    return model

In [11]:
def predict(model, test_X, test_Y):
    predictions = model.predict(test_X)    
    errors = 0
    for i in range(0, predictions.shape[0]):
        for j in range(0, predictions.shape[2], one_hot_size):
            predicted = np.array(encode(decode(predictions[i][0][j:j+one_hot_size]))).astype(int)
            truth = test_Y[i][0][j:j+one_hot_size].astype(int)
            errs = np.sum(np.abs(predicted - truth))/len(truth)
            if debug and errs > 0:
                errors = errors + errs
                print('Predicting ', decode(predicted), '. Truth is ', decode(truth))    
    error_ratio = round((errors/(test_Y.shape[0]*test_Y.shape[2])*100),2)

    return error_ratio

In [12]:
def xp(train_X, train_Y, test_X, test_Y, mode):
    print('xp ', mode)
    results = []
    for i in range(0, N): 
        print('  #', i)
        print('----')
        model = keras_model(train_X[i].shape[2], train_Y[i].shape[2])
        train_start = datetime.datetime.now()
        model = train_and_save(model, train_X[i], train_Y[i], test_X[i], test_Y[i], mode, i, save=True)
        model.summary()
        train_stop = predict_start = datetime.datetime.now()
        error_ratio = predict(model, test_X[i], test_Y[i])
        predict_stop = datetime.datetime.now()
        train_duration = train_stop - train_start
        predict_duration = predict_stop - predict_start
        print(error_ratio, '% error')
        print('training duration = ', train_duration)
        print('prediction duration = ', predict_duration)
        results.append((error_ratio, train_duration, predict_duration))
    return results

In [13]:
base_results = xp(base_X_train_supervised, base_Y_train_supervised, base_X_test_supervised, base_Y_test_supervised, 'base')
static_results = xp(static_X_train_supervised, static_Y_train_supervised, static_X_test_supervised, static_Y_test_supervised, 'static')
runtime_results = xp(runtime_X_train_supervised, runtime_Y_train_supervised, runtime_X_test_supervised, runtime_Y_test_supervised, 'runtime')

xp  base
  # 0
----
Train on 661 samples, validate on 166 samples
Epoch 1/1
661/661 [==============================] - 24s 36ms/step - loss: 0.1055 - acc: 0.9596 - binary_accuracy: 0.9596 - val_loss: 0.0159 - val_acc: 0.9969 - val_binary_accuracy: 0.9969
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri multiple                  2000      
_________________________________________________________________
lstm (LSTM)                  multiple                  120400    
_________________________________________________________________
time_distributed_1 (TimeDist multiple                  5050      
_________________________________________________________________
dense_2 (Dense)              multiple                  1632      
Total params: 129,082
Trainable params: 129,082
Non-trainable params: 0
_________________________________________________________________
Predicting  [

Predicting  ['update', 'x', 'y'] . Truth is  ['update']
Predicting  ['fillRect', 'x', 'y', 'width', 'height'] . Truth is  ['position', 'x', 'y']
Predicting  ['update', 'x', 'y'] . Truth is  ['update']
Predicting  ['fillRect', 'x', 'y', 'width', 'height'] . Truth is  ['position', 'x', 'y']
Predicting  ['update', 'x', 'y'] . Truth is  ['update']
Predicting  ['fillRect', 'x', 'y', 'width', 'height'] . Truth is  ['position', 'x', 'y']
Predicting  ['update', 'x', 'y'] . Truth is  ['update']
Predicting  ['fillRect', 'x', 'y', 'width', 'height'] . Truth is  ['position', 'x', 'y']
Predicting  ['update', 'x', 'y'] . Truth is  ['update']
Predicting  ['fillRect', 'x', 'y', 'width', 'height'] . Truth is  ['position', 'x', 'y']
Predicting  ['update', 'x', 'y'] . Truth is  ['update']
Predicting  ['fillRect', 'x', 'y', 'width', 'height'] . Truth is  ['position', 'x', 'y']
Predicting  ['update', 'x', 'y'] . Truth is  ['update']
Predicting  ['fillRect', 'x', 'y', 'width', 'height'] . Truth is  ['positi

Predicting  ['setColor'] . Truth is  ['setColor', 'g', 'b']
Predicting  ['setColor'] . Truth is  ['setColor', 'r']
Predicting  ['setColor', 'g', 'b'] . Truth is  ['fillRect', 'x', 'y']
Predicting  ['setColor', 'g', 'b'] . Truth is  ['fillRect', 'x', 'y']
Predicting  ['setColor'] . Truth is  ['setColor', 'r']
Predicting  ['fillRect'] . Truth is  ['fillRect', 'width', 'height']
Predicting  ['setColor', 'g', 'b'] . Truth is  ['fillRect', 'x', 'y']
Predicting  ['fillRect'] . Truth is  ['fillRect', 'width', 'height']
Predicting  ['setColor', 'g', 'b'] . Truth is  ['fillRect', 'x', 'y']
Predicting  ['setColor'] . Truth is  ['update', 'padx']
Predicting  ['setColor'] . Truth is  ['update']
Predicting  ['fillRect', 'width', 'height'] . Truth is  ['update']
Predicting  ['fillRect'] . Truth is  ['position', 'y']
Predicting  ['fillRect', 'x', 'y'] . Truth is  ['position', 'x']
Predicting  ['setColor', 'g', 'b', 'x'] . Truth is  ['setColor', 'g', 'b']
Predicting  ['setColor'] . Truth is  ['setColo

Predicting  ['setColor', 'g', 'b', 'x'] . Truth is  ['fillRect', 'x', 'y']
Predicting  ['setColor', 'g', 'b'] . Truth is  ['fillRect', 'x', 'y']
Predicting  ['setColor'] . Truth is  ['setColor', 'r']
Predicting  ['fillRect'] . Truth is  ['fillRect', 'width', 'height']
Predicting  ['setColor', 'g', 'b'] . Truth is  ['fillRect', 'x', 'y']
Predicting  ['fillRect'] . Truth is  ['fillRect', 'width', 'height']
Predicting  ['setColor', 'g', 'b'] . Truth is  ['fillRect', 'x', 'y']
Predicting  ['setColor'] . Truth is  ['update', 'padx']
Predicting  ['setColor', 'r'] . Truth is  ['update']
Predicting  ['fillRect', 'width', 'height'] . Truth is  ['update']
Predicting  ['fillRect'] . Truth is  ['position', 'y']
Predicting  ['setColor', 'x', 'y'] . Truth is  ['position', 'x']
Predicting  ['setColor'] . Truth is  ['setColor', 'r']
Predicting  ['setColor', 'g', 'b'] . Truth is  ['fillRect', 'x', 'y']
Predicting  ['setColor', 'g', 'b'] . Truth is  ['fillRect', 'x', 'y']
Predicting  ['setColor'] . Trut

Predicting  ['update'] . Truth is  ['fillRect', 'width', 'height']
Predicting  ['update', 'width', 'height'] . Truth is  ['update']
Predicting  ['setColor'] . Truth is  ['update']
Predicting  ['update'] . Truth is  ['position']
Predicting  ['setColor', 'x'] . Truth is  ['position', 'x', 'y']
Predicting  ['setColor'] . Truth is  ['fillRect', 'x', 'width', 'height']
Predicting  ['setColor'] . Truth is  ['setColor', 'r', 'b']
Predicting  ['setColor', 'r'] . Truth is  ['setColor', 'g']
Predicting  ['update', 'height'] . Truth is  ['fillRect', 'width', 'height']
Predicting  ['setColor', 'g', 'b'] . Truth is  ['fillRect', 'x', 'y']
Predicting  ['setColor', 'b'] . Truth is  ['setColor', 'r', 'b']
Predicting  ['fillRect', 'height'] . Truth is  ['setColor', 'g']
Predicting  ['setColor', 'g', 'b'] . Truth is  ['fillRect', 'x', 'y']
Predicting  ['update'] . Truth is  ['fillRect', 'width', 'height']
Predicting  ['fillRect'] . Truth is  ['fillRect', 'x', 'width', 'height']
Predicting  ['update', 'w

Predicting  ['setColor'] . Truth is  ['setColor', 'g']
Predicting  ['setColor', 'g', 'b'] . Truth is  ['setColor', 'r', 'b']
Predicting  ['update'] . Truth is  ['fillRect', 'width', 'height']
Predicting  ['setColor', 'g', 'b'] . Truth is  ['fillRect', 'x', 'y']
Predicting  ['update', 'width', 'height'] . Truth is  ['update']
Predicting  ['setColor'] . Truth is  ['update']
Predicting  ['setColor', 'x'] . Truth is  ['position', 'x', 'y']
Predicting  ['update'] . Truth is  ['position']
0.2 % error
training duration =  0:00:49.523795
prediction duration =  0:00:01.075424
